<a href="https://colab.research.google.com/github/CodeandoMexico/hacking-civico/blob/master/notebooks/05_Exploracion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="http://codeandomexico.org/resources/img/codeandomexico.png" width="500" alt="Codeando México"><br>
<a href="http://www.codeandomexico.org/" target="_blank"><img src="https://img.shields.io/badge/website-CodeandoMexico-00D88E.svg"></a>
<a href="http://slack.codeandomexico.org/" target="_blank"><img src="https://img.shields.io/badge/slack-CodeandoMexico-EC0E4F.svg"></a>
</p>
<!-- __ -->

# Curso de Datos Abiertos y Hacking Cívico

Este curso tiene como objetivo habilitar las capacidades de la ciudadanía y los servidores públicos en el uso y generación de datos abiertos para el bien común. Puedes encontrar más información [aquí](https://github.com/CodeandoMexico/hacking-civico).

# Exploración de datos: COVID-19 en México

En este cuaderno aprenderemos algunas bases de exploración de datos. Los datos a utilizar son los datos abiertos sobre COVID-19 en México, los cuales pueden ser accedidos directamente desde el portal: https://www.gob.mx/salud/documentos/datos-abiertos-152127

Del portal de datos abiertos podemos encontrar dos enlaces, uno para descargar el conjunto de datos y otro para descargar el diccionario; así que podemos automatizar el proceso de descarga y descompresión de la información.

Para obtener los datos, sólo basta que ejecutes la siguiente línea de código, que contiene el comando de descarga de la información.

In [1]:
# Para descargar el conjunto de datos
!wget http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

# Para descargar el diccionario de los datos
!wget http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip

# Descomprimimos ambas carpetas y movemos archivos
!unzip datos_abiertos_covid19.zip
!unzip diccionario_datos_covid19.zip
!mv ./diccionario_datos_covid19/*.xlsx .
!mv *COVID19MEXICO.csv COVID19MEXICO.csv

# Eliminamos las carpetas comprimidas
!rm -rf diccionario_datos_covid19
!rm datos_abiertos_covid19.zip
!rm diccionario_datos_covid19.zip

--2020-08-31 04:25:22--  http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 187.210.186.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|187.210.186.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28367152 (27M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  27.05M  14.8MB/s    in 1.8s    

2020-08-31 04:25:24 (14.8 MB/s) - ‘datos_abiertos_covid19.zip’ saved [28367152/28367152]

--2020-08-31 04:25:24--  http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip
Resolving epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)... 187.191.75.207
Connecting to epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)|187.191.75.207|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ep

Hasta este punto, si abres la barra lateral izaquiera, deberías ser capaz de encontrar los datos:

- `COVID19MEXICO.csv`
- `Catalogos_0412.xlsx`
- `Descriptores_0419.xlsx`

👈🏼

## Carga

Ahora que tenemos los datos, procederemos a cargarlos. Para cargar los datos, primero importaremos algunos paquetes de Python, pues su funcionalidad nos será muy util.

In [2]:
import statistics

import pandas as pd
import numpy as np

In [3]:
# Especificamos el nombre del archivo por abrir
nombre_archivo = 'COVID19MEXICO.csv'

# Utilizamos la función de Pandas para cargar un archivo CSV
data = pd.read_csv(nombre_archivo, encoding='latin-1')

Mostramos la cabeza (primeros elementos del archivo).

In [4]:
data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-08-30,1b1466,2,4,23,2,18,23,5,1,2020-04-13,2020-04-12,9999-99-99,97,2,38,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-08-30,14b4f8,2,4,30,1,30,30,131,1,2020-04-17,2020-04-15,9999-99-99,97,2,50,1,2,2,2,2,2,1,2,1,2,2,2,2,99,1,99,MÃ©xico,99,97
2,2020-08-30,082c0d,2,4,9,2,9,9,17,1,2020-04-12,2020-04-07,9999-99-99,97,2,50,1,97,2,2,2,2,2,1,2,2,1,2,2,1,1,99,MÃ©xico,99,97
3,2020-08-30,152344,2,4,15,1,15,15,58,2,2020-04-26,2020-04-26,9999-99-99,2,2,39,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2
4,2020-08-30,08fe2c,2,3,15,2,15,15,106,1,2020-05-30,2020-05-25,9999-99-99,97,1,26,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97


Ahora procedemos a cargar el archivo con los descriptores.

In [5]:
nombre_diccionario = 'Descriptores_0419.xlsx'
diccionario = pd.read_excel(nombre_diccionario)

In [6]:
diccionario

,Nº,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE,FORMATO O FUENTE
0,1,FECHA_ACTUALIZACION,"La base de datos se alimenta diariamente, esta...",AAAA-MM-DD
1,2,ID_REGISTRO,Número identificador del caso,TEXTO
2,3,ORIGEN,La vigilancia centinela se realiza a través de...,CATÁLOGO: ORIGEN ...
3,4,SECTOR,Identifica el tipo de institución del Sistema ...,CATÁLOGO: SECTOR ...
4,5,ENTIDAD_UM,Identifica la entidad donde se ubica la unidad...,CATALÓGO: ENTIDADES
5,6,SEXO,Identifica al sexo del paciente.,CATÁLOGO: SEXO
6,7,ENTIDAD_NAC,Identifica la entidad de nacimiento del paciente.,CATALÓGO: ENTIDADES
7,8,ENTIDAD_RES,Identifica la entidad de residencia del paciente.,CATALÓGO: ENTIDADES
8,9,MUNICIPIO_RES,Identifica el municipio de residencia del paci...,CATALÓGO: MUNICIPIOS
9,10,TIPO_PACIENTE,Identifica el tipo de atención que recibió el ...,CATÁLOGO: TIPO_PACIENTE


Finalmente, cargamos el rachivo de catálogos.

In [7]:
nombre_cat = 'Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(nombre_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

In [8]:
catalogues

{'MUNICIPIOS':       clave_municipio        municipio  clave_entidad
 0                   1   AGUASCALIENTES              1
 1                   2         ASIENTOS              1
 2                   3         CALVILLO              1
 3                   4            COSÍO              1
 4                   5      JESÚS MARÍA              1
 ...               ...              ...            ...
 2496              999  NO ESPECIFICADO             32
 2497              999  NO ESPECIFICADO             36
 2498              997        NO APLICA             97
 2499              998        SE IGNORA             98
 2500              999  NO ESPECIFICADO             99
 
 [2501 rows x 3 columns], 'NACIONALIDAD':    clave      descripción
 0      1         MEXICANA
 1      2       EXTRANJERA
 2     99  NO ESPECIFICADO, 'ORIGEN':    clave      descripción
 0      1            USMER
 1      2   FUERA DE USMER
 2     99  NO ESPECIFICADO, 'RESULTADO':   unnamed:_0              unnamed:_1
 0    

In [9]:
catalogues['SEXO']

,clave,descripción
0,1,MUJER
1,2,HOMBRE
2,99,NO ESPECIFICADO


##Perfilamiento

###Descripción General

In [10]:
# dimensión de los datos
print('Número de filas: {}'.format(data.shape[0]))
print('Número de columnas: {}'.format(data.shape[1]))

Número de filas: 1340466
Número de columnas: 35


In [11]:
# columnas (variables)
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [12]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [13]:
# calcular número de valores nulos por columna
for col in data.columns:
  print('Valores nulos en "{}": {}'.format(col,data[col].isna().sum()))

Valores nulos en "fecha_actualizacion": 0
Valores nulos en "id_registro": 0
Valores nulos en "origen": 0
Valores nulos en "sector": 0
Valores nulos en "entidad_um": 0
Valores nulos en "sexo": 0
Valores nulos en "entidad_nac": 0
Valores nulos en "entidad_res": 0
Valores nulos en "municipio_res": 0
Valores nulos en "tipo_paciente": 0
Valores nulos en "fecha_ingreso": 0
Valores nulos en "fecha_sintomas": 0
Valores nulos en "fecha_def": 0
Valores nulos en "intubado": 0
Valores nulos en "neumonia": 0
Valores nulos en "edad": 0
Valores nulos en "nacionalidad": 0
Valores nulos en "embarazo": 0
Valores nulos en "habla_lengua_indig": 0
Valores nulos en "diabetes": 0
Valores nulos en "epoc": 0
Valores nulos en "asma": 0
Valores nulos en "inmusupr": 0
Valores nulos en "hipertension": 0
Valores nulos en "otra_com": 0
Valores nulos en "cardiovascular": 0
Valores nulos en "obesidad": 0
Valores nulos en "renal_cronica": 0
Valores nulos en "tabaquismo": 0
Valores nulos en "otro_caso": 0
Valores nulos 

In [14]:
# tipos de datos
data.dtypes

fecha_actualizacion    object
id_registro            object
origen                  int64
sector                  int64
entidad_um              int64
sexo                    int64
entidad_nac             int64
entidad_res             int64
municipio_res           int64
tipo_paciente           int64
fecha_ingreso          object
fecha_sintomas         object
fecha_def              object
intubado                int64
neumonia                int64
edad                    int64
nacionalidad            int64
embarazo                int64
habla_lengua_indig      int64
diabetes                int64
epoc                    int64
asma                    int64
inmusupr                int64
hipertension            int64
otra_com                int64
cardiovascular          int64
obesidad                int64
renal_cronica           int64
tabaquismo              int64
otro_caso               int64
resultado               int64
migrante                int64
pais_nacionalidad      object
pais_orige

###Limpieza

In [15]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [16]:
data.head()

,fecha_actualizacion,id_registro,origen,sector,entidad_um,sexo,entidad_nac,entidad_res,municipio_res,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,edad,nacionalidad,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,resultado,migrante,pais_nacionalidad,pais_origen,uci
0,2020-08-30,1b1466,2,4,23,2,18,23,5,1,2020-04-13,2020-04-12,NaT,97,2,38,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-08-30,14b4f8,2,4,30,1,30,30,131,1,2020-04-17,2020-04-15,NaT,97,2,50,1,2,2,2,2,2,1,2,1,2,2,2,2,99,1,99,MÃ©xico,99,97
2,2020-08-30,082c0d,2,4,9,2,9,9,17,1,2020-04-12,2020-04-07,NaT,97,2,50,1,97,2,2,2,2,2,1,2,2,1,2,2,1,1,99,MÃ©xico,99,97
3,2020-08-30,152344,2,4,15,1,15,15,58,2,2020-04-26,2020-04-26,NaT,2,2,39,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2
4,2020-08-30,08fe2c,2,3,15,2,15,15,106,1,2020-05-30,2020-05-25,NaT,97,1,26,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97


In [17]:
# unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

In [18]:
# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [19]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [ ]:
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [21]:
# creamos algunas variables que podrían ser útiles después
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

In [22]:
data.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-08-30,1b1466,2020-04-13,2020-04-12,NaT,38,1,MÃ©xico,99,QUINTANA ROO,BENITO JUÁREZ,QUINTANA ROO,NAYARIT,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
1,2020-08-30,14b4f8,2020-04-17,2020-04-15,NaT,50,1,MÃ©xico,99,VERACRUZ DE IGNACIO DE LA LLAVE,POZA RICA DE HIDALGO,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,NO APLICA,NO,NO,NO,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
2,2020-08-30,082c0d,2020-04-12,2020-04-07,NaT,50,1,MÃ©xico,99,CIUDAD DE MÉXICO,VENUSTIANO CARRANZA,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
3,2020-08-30,152344,2020-04-26,2020-04-26,NaT,39,1,MÃ©xico,99,MÉXICO,NEZAHUALCÓYOTL,MÉXICO,MÉXICO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,False,NaN,NaN,4,2020
4,2020-08-30,08fe2c,2020-05-30,2020-05-25,NaT,26,1,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,MÉXICO,NO APLICA,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,5,2020


###Filtros

**Distintas formas de filtrar lo mismo**

In [23]:
# usando query
data.query('nacionalidad == "MEXICANA"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-08-30,1b1466,2020-04-13,2020-04-12,NaT,38,1,MÃ©xico,99,QUINTANA ROO,BENITO JUÁREZ,QUINTANA ROO,NAYARIT,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
1,2020-08-30,14b4f8,2020-04-17,2020-04-15,NaT,50,1,MÃ©xico,99,VERACRUZ DE IGNACIO DE LA LLAVE,POZA RICA DE HIDALGO,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,NO APLICA,NO,NO,NO,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020


In [24]:
# haciendo referencia a la columna como índice
data[data['nacionalidad'] == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-08-30,1b1466,2020-04-13,2020-04-12,NaT,38,1,MÃ©xico,99,QUINTANA ROO,BENITO JUÁREZ,QUINTANA ROO,NAYARIT,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
1,2020-08-30,14b4f8,2020-04-17,2020-04-15,NaT,50,1,MÃ©xico,99,VERACRUZ DE IGNACIO DE LA LLAVE,POZA RICA DE HIDALGO,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,NO APLICA,NO,NO,NO,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020


In [25]:
# seleccionando directamente la columna que queremos filtrar
data[data.nacionalidad == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-08-30,1b1466,2020-04-13,2020-04-12,NaT,38,1,MÃ©xico,99,QUINTANA ROO,BENITO JUÁREZ,QUINTANA ROO,NAYARIT,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
1,2020-08-30,14b4f8,2020-04-17,2020-04-15,NaT,50,1,MÃ©xico,99,VERACRUZ DE IGNACIO DE LA LLAVE,POZA RICA DE HIDALGO,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,NO APLICA,NO,NO,NO,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020


**Filtrado múltiple**

In [26]:
# usando query podemos poner en el string del query todas las condiciones que queramos
data.query('nacionalidad == "MEXICANA" & entidad_res == "CIUDAD DE MÉXICO"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
2,2020-08-30,082c0d,2020-04-12,2020-04-07,NaT,50,1,MÃ©xico,99,CIUDAD DE MÉXICO,VENUSTIANO CARRANZA,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
8,2020-08-30,0aa443,2020-04-28,2020-04-23,NaT,43,1,MÃ©xico,99,CIUDAD DE MÉXICO,VENUSTIANO CARRANZA,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020


In [27]:
# usando el filtrado tradicional debemos encerrar en paréntesis cada condición que queremos que cumpla nuestro filtro
data[(data['nacionalidad'] == "MEXICANA") & (data.entidad_res == "CIUDAD DE MÉXICO")].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
2,2020-08-30,082c0d,2020-04-12,2020-04-07,NaT,50,1,MÃ©xico,99,CIUDAD DE MÉXICO,VENUSTIANO CARRANZA,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
8,2020-08-30,0aa443,2020-04-28,2020-04-23,NaT,43,1,MÃ©xico,99,CIUDAD DE MÉXICO,VENUSTIANO CARRANZA,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020


###Resúmenes

In [28]:
# una columna, una métrica
data.groupby('sexo').edad.mean()

sexo
HOMBRE    42.858285
MUJER     41.768781
Name: edad, dtype: float64

In [29]:
# una columna, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max']})

edad                
        min       mean  max
sexo                       
HOMBRE    0  42.858285  120
MUJER     0  41.768781  120

In [30]:
# varias columnas, una métrica
data.groupby('sexo').agg({'edad': 'mean', 'mes_ingreso': 'mean', 'mes_def': 'mean'})

,edad,mes_ingreso,mes_def
sexo,,,
HOMBRE,42.858285,6.472954,6.314927
MUJER,41.768781,6.533079,6.373319


In [31]:
# varias columnas, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max'], 'mes_ingreso': ['min','mean','max'], 'mes_def': ['min','mean','max']})

edad                 mes_ingreso               mes_def               
        min       mean  max         min      mean max     min      mean  max
sexo                                                                        
HOMBRE    0  42.858285  120           1  6.472954   8     1.0  6.314927  8.0
MUJER     0  41.768781  120           1  6.533079   8     1.0  6.373319  8.0

###Descripción por Tipo

In [32]:
# Filtrar por tipos generales (categóricas y numéricas)
numerical = data.select_dtypes(include='number')
categorical = data.select_dtypes(include=['object', 'bool', 'category', 'datetime64[ns]'])

In [33]:
numerical.head()

,edad,resultado,mes_def,año_def,mes_ingreso,año_ingreso
0,38,1,NaN,NaN,4,2020
1,50,1,NaN,NaN,4,2020
2,50,1,NaN,NaN,4,2020
3,39,1,NaN,NaN,4,2020
4,26,1,NaN,NaN,5,2020


In [34]:
categorical.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion
0,2020-08-30,1b1466,2020-04-13,2020-04-12,NaT,MÃ©xico,99,QUINTANA ROO,BENITO JUÁREZ,QUINTANA ROO,NAYARIT,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False
1,2020-08-30,14b4f8,2020-04-17,2020-04-15,NaT,MÃ©xico,99,VERACRUZ DE IGNACIO DE LA LLAVE,POZA RICA DE HIDALGO,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,NO APLICA,NO,NO,NO,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False
2,2020-08-30,082c0d,2020-04-12,2020-04-07,NaT,MÃ©xico,99,CIUDAD DE MÉXICO,VENUSTIANO CARRANZA,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False
3,2020-08-30,152344,2020-04-26,2020-04-26,NaT,MÃ©xico,99,MÉXICO,NEZAHUALCÓYOTL,MÉXICO,MÉXICO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,False
4,2020-08-30,08fe2c,2020-05-30,2020-05-25,NaT,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,MÉXICO,NO APLICA,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,False


In [35]:
# para conocer todos los tipo de datos
?data.select_dtypes

####Categóricas

In [36]:
# Cardinalidad - Valores únicos por columna
for col in categorical.columns:
  print('Valores únicos en "{}": {}'.format(col,categorical[col].nunique()))

Valores únicos en "fecha_actualizacion": 1
Valores únicos en "id_registro": 1340466
Valores únicos en "fecha_ingreso": 243
Valores únicos en "fecha_sintomas": 243
Valores únicos en "fecha_def": 188
Valores únicos en "pais_nacionalidad": 117
Valores únicos en "pais_origen": 77
Valores únicos en "entidad_res": 32
Valores únicos en "municipio_res": 2149
Valores únicos en "entidad_um": 32
Valores únicos en "entidad_nac": 33
Valores únicos en "intubado": 4
Valores únicos en "neumonia": 3
Valores únicos en "embarazo": 4
Valores únicos en "habla_lengua_indig": 3
Valores únicos en "diabetes": 3
Valores únicos en "epoc": 3
Valores únicos en "asma": 3
Valores únicos en "inmusupr": 3
Valores únicos en "hipertension": 3
Valores únicos en "otra_com": 3
Valores únicos en "cardiovascular": 3
Valores únicos en "obesidad": 3
Valores únicos en "renal_cronica": 3
Valores únicos en "tabaquismo": 3
Valores únicos en "otro_caso": 3
Valores únicos en "migrante": 3
Valores únicos en "uci": 4
Valores únicos en

In [37]:
# Unicidad - proporción de valores únicos por columna
total = data.shape[0]
for col in categorical.columns:
  print('Porcentaje de valores únicos en "{}": {}%'.format(col,round(categorical[col].nunique()/total*100,2)))

Porcentaje de valores únicos en "fecha_actualizacion": 0.0%
Porcentaje de valores únicos en "id_registro": 100.0%
Porcentaje de valores únicos en "fecha_ingreso": 0.02%
Porcentaje de valores únicos en "fecha_sintomas": 0.02%
Porcentaje de valores únicos en "fecha_def": 0.01%
Porcentaje de valores únicos en "pais_nacionalidad": 0.01%
Porcentaje de valores únicos en "pais_origen": 0.01%
Porcentaje de valores únicos en "entidad_res": 0.0%
Porcentaje de valores únicos en "municipio_res": 0.16%
Porcentaje de valores únicos en "entidad_um": 0.0%
Porcentaje de valores únicos en "entidad_nac": 0.0%
Porcentaje de valores únicos en "intubado": 0.0%
Porcentaje de valores únicos en "neumonia": 0.0%
Porcentaje de valores únicos en "embarazo": 0.0%
Porcentaje de valores únicos en "habla_lengua_indig": 0.0%
Porcentaje de valores únicos en "diabetes": 0.0%
Porcentaje de valores únicos en "epoc": 0.0%
Porcentaje de valores únicos en "asma": 0.0%
Porcentaje de valores únicos en "inmusupr": 0.0%
Porcenta

In [38]:
# Moda - valor que más se repite en cada columna excepto los indicadores únicos
for col in categorical.drop(columns='id_registro').columns:
  print('Valor más popular en "{}": {}'.format(col, statistics.mode(categorical[col])))

Valor más popular en "fecha_actualizacion": 2020-08-30 00:00:00
Valor más popular en "fecha_ingreso": 2020-07-20 00:00:00
Valor más popular en "fecha_sintomas": 2020-07-20 00:00:00
Valor más popular en "fecha_def": NaT
Valor más popular en "pais_nacionalidad": MÃ©xico
Valor más popular en "pais_origen": 99
Valor más popular en "entidad_res": CIUDAD DE MÉXICO
Valor más popular en "municipio_res": IZTAPALAPA
Valor más popular en "entidad_um": CIUDAD DE MÉXICO
Valor más popular en "entidad_nac": CIUDAD DE MÉXICO
Valor más popular en "intubado": NO APLICA
Valor más popular en "neumonia": NO 
Valor más popular en "embarazo": NO 
Valor más popular en "habla_lengua_indig": NO 
Valor más popular en "diabetes": NO 
Valor más popular en "epoc": NO 
Valor más popular en "asma": NO 
Valor más popular en "inmusupr": NO 
Valor más popular en "hipertension": NO 
Valor más popular en "otra_com": NO 
Valor más popular en "cardiovascular": NO 
Valor más popular en "obesidad": NO 
Valor más popular en "r

####Numéricas

In [39]:
numerical.describe()

,edad,resultado,mes_def,año_def,mes_ingreso,año_ingreso
count,1.340466e+06,1.340466e+06,81778.000000,81778.0,1.340466e+06,1340466.0
mean,4.230687e+01,1.616037e+00,6.336117,2020.0,6.503384e+00,2020.0
std,1.676442e+01,5.980088e-01,1.193375,0.0,1.248783e+00,0.0
min,0.000000e+00,1.000000e+00,1.000000,2020.0,1.000000e+00,2020.0
25%,3.000000e+01,1.000000e+00,5.000000,2020.0,6.000000e+00,2020.0
50%,4.100000e+01,2.000000e+00,6.000000,2020.0,7.000000e+00,2020.0
75%,5.300000e+01,2.000000e+00,7.000000,2020.0,8.000000e+00,2020.0
max,1.200000e+02,3.000000e+00,8.000000,2020.0,8.000000e+00,2020.0


##Análisis Exploratorio

En esta sección responderemos algunas preguntas sobre el set de datos usando las herramientas que hemos visto hasta ahora

In [40]:
# ¿Cuántas pruebas por origen se realizan?
data.groupby('origen').id_registro.count()

origen
FUERA DE USMER    891500
USMER             448966
Name: id_registro, dtype: int64

In [41]:
data.origen.value_counts()

FUERA DE USMER    891500
USMER             448966
Name: origen, dtype: int64

In [42]:
# ¿Cuántas pruebas por sector se realizan?
data.sector.value_counts()

SSA                829327
IMSS               346673
ISSSTE              51064
PRIVADA             50313
ESTATAL             25105
PEMEX               11896
IMSS-BIENESTAR       9367
SEMAR                7160
SEDENA               6618
MUNICIPAL            1108
UNIVERSITARIO         864
DIF                   768
CRUZ ROJA             195
NO ESPECIFICADO         8
Name: sector, dtype: int64

In [43]:
# ¿Cuántos pacientes por sexo tenemos?
data.sexo.value_counts()

MUJER     678427
HOMBRE    662039
Name: sexo, dtype: int64

In [44]:
# ¿Cuál es la proporción de los pacientes por sexo?
data.sexo.value_counts(normalize=True)

MUJER     0.506113
HOMBRE    0.493887
Name: sexo, dtype: float64

In [45]:
# ¿Cuál es la proporción de resultados de las pruebas?
data.resultado.value_counts(normalize=True)

2    0.494958
1    0.444503
3    0.060539
Name: resultado, dtype: float64

In [46]:
# ¿Cuántas pruebas positivas tenemos en el set?
data[data.resultado == 'Positivo SARS-CoV-2'].id_registro.count()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


0

In [47]:
# ¿Cuántos casos positivos tenemos por mes?
data[data.resultado == 'Positivo SARS-CoV-2'].mes_ingreso.value_counts().sort_index()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Series([], Name: mes_ingreso, dtype: int64)

In [48]:
# ¿Cuál es el resultado de las pruebas por sexo?
data.groupby(['sexo','resultado'])['id_registro'].count()

sexo    resultado
HOMBRE  1            312730
        2            309379
        3             39930
MUJER   1            283111
        2            354095
        3             41221
Name: id_registro, dtype: int64

In [49]:
data.groupby('sexo').resultado.value_counts()

sexo    resultado
HOMBRE  1            312730
        2            309379
        3             39930
MUJER   2            354095
        1            283111
        3             41221
Name: resultado, dtype: int64

In [50]:
# ¿Qué proporción de resultados de pruebas tenemos por sexo?
data.groupby('sexo').resultado.value_counts(normalize=True)

sexo    resultado
HOMBRE  1            0.472374
        2            0.467312
        3            0.060314
MUJER   2            0.521935
        1            0.417305
        3            0.060760
Name: resultado, dtype: float64

In [51]:
# ¿Cuál es la edad de los pacientes por sexo?
data.groupby('sexo').edad.describe()

,count,mean,std,min,25%,50%,75%,max
sexo,,,,,,,,
HOMBRE,662039.0,42.858285,17.069314,0.0,30.0,41.0,54.0,120.0
MUJER,678427.0,41.768781,16.443640,0.0,30.0,40.0,52.0,120.0


In [52]:
def q25(x):
  return x.quantile(0.25)

def q75(x):
  return x.quantile(0.75)

data.groupby('sexo').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                    
        min q25 median q75  max
sexo                           
HOMBRE    0  30     41  54  120
MUJER     0  30     40  52  120

In [53]:
# ¿Cuál es la edad de los pacientes dependiendo de si son intubados o no?
data.groupby('intubado').edad.describe()

,count,mean,std,min,25%,50%,75%,max
intubado,,,,,,,,
NO,213244.0,53.612435,19.404141,0.0,42.0,55.0,67.0,120.0
NO APLICA,1099016.0,39.770084,14.994031,0.0,29.0,38.0,49.0,120.0
NO ESPECIFICADO,171.0,53.315789,17.778200,0.0,42.0,54.0,66.0,92.0
SI,28035.0,55.691778,18.824723,0.0,47.0,58.0,68.0,108.0


In [54]:
data.groupby('intubado').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                    
                 min q25 median q75  max
intubado                                
NO                 0  42     55  67  120
NO APLICA          0  29     38  49  120
NO ESPECIFICADO    0  42     54  66   92
SI                 0  47     58  68  108

In [55]:
# ¿Cuál es la edad máxima y cuál es la mínima de los pacientes fallecidos?
edad_def_max = data[data.defuncion].edad.max()
data[data.defuncion & (data.edad == edad_def_max)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
618302,2020-08-30,1b7b20,2020-05-21,2020-05-20,2020-05-26,109,2,MÃ©xico,99,MICHOACÁN DE OCAMPO,APATZINGÁN,MICHOACÁN DE OCAMPO,MICHOACÁN DE OCAMPO,NO,SI,NO APLICA,NO,NO,NO,NO,SI,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,True,5.0,2020.0,5,2020


In [56]:
edad_def_min = data[data.defuncion].edad.min()
data[data.defuncion & (data.edad == edad_def_min)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
12016,2020-08-30,18feee,2020-06-23,2020-06-23,2020-07-06,0,1,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,True,7.0,2020.0,6,2020
14070,2020-08-30,02f8e2,2020-05-08,2020-05-01,2020-05-10,0,1,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,True,5.0,2020.0,5,2020
21220,2020-08-30,083c34,2020-05-25,2020-05-25,2020-06-01,0,1,MÃ©xico,99,MÉXICO,CHIMALHUACÁN,MÉXICO,MÉXICO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,True,6.0,2020.0,5,2020
37677,2020-08-30,1cc0ea,2020-04-24,2020-04-21,2020-04-29,0,1,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,MÉXICO,MÉXICO,NO,SI,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,SSA,MUJER,HOSPITALIZADO,MEXICANA,True,4.0,2020.0,4,2020
40605,2020-08-30,053398,2020-06-11,2020-06-11,2020-06-17,0,1,MÃ©xico,99,AGUASCALIENTES,AGUASCALIENTES,AGUASCALIENTES,AGUASCALIENTES,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,True,6.0,2020.0,6,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264520,2020-08-30,0b9e95,2020-07-02,2020-07-02,2020-07-07,0,2,MÃ©xico,99,OAXACA,SANTA LUCÍA DEL CAMINO,OAXACA,OAXACA,SI,NO,NO APLICA,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,True,7.0,2020.0,7,2020
1265332,2020-08-30,095e15,2020-07-03,2020-07-01,2020-07-04,0,2,MÃ©xico,99,MICHOACÁN DE OCAMPO,MORELIA,MICHOACÁN DE OCAMPO,MICHOACÁN DE OCAMPO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,SI,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,True,7.0,2020.0,7,2020
1270388,2020-08-30,1d807b,2020-08-11,2020-08-10,2020-08-11,0,2,MÃ©xico,99,GUERRERO,TLAPA DE COMONFORT,GUERRERO,GUERRERO,NO,SI,NO APLICA,NO ESPECIFICADO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO,FUERA DE USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,True,8.0,2020.0,8,2020
1276457,2020-08-30,0860f4,2020-06-30,2020-06-30,2020-07-13,0,3,MÃ©xico,99,PUEBLA,TEHUACÁN,PUEBLA,PUEBLA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,True,7.0,2020.0,6,2020


In [57]:
# ¿Cuál es la edad de los pacientes que fallecen?
data.groupby('defuncion').edad.describe()

,count,mean,std,min,25%,50%,75%,max
defuncion,,,,,,,,
False,1258688.0,41.042617,16.064551,0.0,30.0,40.0,51.0,120.0
True,81778.0,61.765707,15.213004,0.0,53.0,63.0,72.0,109.0


In [58]:
# ¿Cuántas defunciones tenemos por mes?
data[data.defuncion].mes_def.value_counts().sort_index()

1.0        6
2.0       12
3.0      232
4.0     5257
5.0    15988
6.0    21266
7.0    23271
8.0    15746
Name: mes_def, dtype: int64

In [59]:
# ¿Cuál es la proporción de diábeticos para las defunciones?
data.groupby('diabetes').defuncion.value_counts(normalize=True)

diabetes   defuncion
NO         False        0.957060
           True         0.042940
SE IGNORA  False        0.871588
           True         0.128412
SI         False        0.812516
           True         0.187484
Name: defuncion, dtype: float64

In [60]:
# ¿Cuál es la proporción de asmáticos para las defunciones?
data.groupby('asma').defuncion.value_counts(normalize=True)

asma       defuncion
NO         False        0.938643
           True         0.061357
SE IGNORA  False        0.862844
           True         0.137156
SI         False        0.957260
           True         0.042740
Name: defuncion, dtype: float64